In [1]:
# Sirket_Harcama_Kar_Bilgileri.csv 
# veriseti çoklu regresyon

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/erkansirin78/datasets/master/Sirket_Harcama_Kar_Bilgileri.csv")

# 1. Veri Keşfi

In [5]:
df.head()

ArgeHarcamasi  YonetimGiderleri  PazarlamaHarcamasi     Sehir        Kar
0      165349.20         136897.80           471784.10  Istanbul  192261.83
1      162597.70         151377.59           443898.53    Ankara  191792.06
2      153441.51         101145.55           407934.54   Kocaeli  191050.39
3      144372.41         118671.85           383199.62  Istanbul  182901.99
4      142107.34          91391.77           366168.42   Kocaeli  166187.94

## 1.1. Veri seti şemasını, boş değer sayısını görme

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
ArgeHarcamasi         50 non-null float64
YonetimGiderleri      50 non-null float64
PazarlamaHarcamasi    50 non-null float64
Sehir                 50 non-null object
Kar                   50 non-null float64
dtypes: float64(4), object(1)
memory usage: 2.0+ KB


## 1.2. Histogram

In [8]:
df.hist(bins=15, figsize=(20,15))
plt.show()

## 1.3. Korelasyonlar

In [9]:
df.corr()

ArgeHarcamasi  YonetimGiderleri  PazarlamaHarcamasi  \
ArgeHarcamasi            1.000000          0.241955            0.724248   
YonetimGiderleri         0.241955          1.000000           -0.032154   
PazarlamaHarcamasi       0.724248         -0.032154            1.000000   
Kar                      0.972900          0.200717            0.747766   

                         Kar  
ArgeHarcamasi       0.972900  
YonetimGiderleri    0.200717  
PazarlamaHarcamasi  0.747766  
Kar                 1.000000

In [11]:
# Girdi değişkenlerin hedef değişken ile korelasyonları
df.corrwith(df['Kar']).sort_values(ascending=False)

Kar                   1.000000
ArgeHarcamasi         0.972900
PazarlamaHarcamasi    0.747766
YonetimGiderleri      0.200717
dtype: float64

## 1.4. Nümerik nitelikleri inceleme

In [12]:
df.describe().T

count         mean            std       min          25%  \
ArgeHarcamasi        50.0   73721.6156   45902.256482      0.00   39936.3700   
YonetimGiderleri     50.0  121344.6396   28017.802755  51283.14  103730.8750   
PazarlamaHarcamasi   50.0  211025.0978  122290.310726      0.00  129300.1325   
Kar                  50.0  112012.6392   40306.180338  14681.40   90138.9025   

                           50%          75%        max  
ArgeHarcamasi        73051.080  101602.8000  165349.20  
YonetimGiderleri    122699.795  144842.1800  182645.56  
PazarlamaHarcamasi  212716.240  299469.0850  471784.10  
Kar                 107978.190  139765.9775  192261.83

## 1.5. Sütunları yeniden sıralama

In [54]:
# İstediğim gibi değiştirmiyorum

# 2. Veri Temizliği ve Ön Hazırlığı

## 2.1. Kayıp veri var mı?

In [13]:
df.isnull().sum()

ArgeHarcamasi         0
YonetimGiderleri      0
PazarlamaHarcamasi    0
Sehir                 0
Kar                   0
dtype: int64

## 2.2. SimpleImputer ile total_bedrooms niteliğindeki boşları doldurma

In [14]:
# Boş değerim yok

In [15]:
df.head()

ArgeHarcamasi  YonetimGiderleri  PazarlamaHarcamasi     Sehir        Kar
0      165349.20         136897.80           471784.10  Istanbul  192261.83
1      162597.70         151377.59           443898.53    Ankara  191792.06
2      153441.51         101145.55           407934.54   Kocaeli  191050.39
3      144372.41         118671.85           383199.62  Istanbul  182901.99
4      142107.34          91391.77           366168.42   Kocaeli  166187.94

## 2.3. Veriyi X,y olarak bölme (nitelikler matrisi ve hedef değişken - dikey bölme) aka Numpy Dünyasına Giriş 

In [16]:
# Hedef değişken artık en sonda
X = df.iloc[:,:-1].values
X.shape

(50, 4)

In [17]:
y = df.iloc[:,-1].values
y.shape

(50,)

## 2.4. LabelEncoder ile kategorik nitelikleri kodlama

In [18]:
from sklearn.preprocessing import LabelEncoder
label_encoder_sehir = LabelEncoder()

In [20]:
X[:3,:]

array([[165349.2, 136897.8, 471784.1, 'Istanbul'],
       [162597.7, 151377.59, 443898.53, 'Ankara'],
       [153441.51, 101145.55, 407934.54, 'Kocaeli']], dtype=object)

In [21]:
X[:,3] = label_encoder_sehir.fit_transform(X[:,3])
X.shape

(50, 4)

In [22]:
X[:3,:]

array([[165349.2, 136897.8, 471784.1, 1],
       [162597.7, 151377.59, 443898.53, 0],
       [153441.51, 101145.55, 407934.54, 2]], dtype=object)

## 2.5. OneHotEncoder ile kodlanan kategorik nitelikleri gölge değişkenler yapma

In [23]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder(handle_unknown='ignore', categories='auto')

In [25]:
# Tek bir kategorik nitalik var. Acaba kaç farklı kategori var?
len(df.Sehir.unique())

3

In [26]:
# Bir veya daha fazla sütunu ilgilendiren dönüşümleri kolay yapabilmek için kullanışlı bir sınıf (Estimator kökenli)
# Biz burada sadece OneHotEncoder için kullanacağız.
from sklearn.compose import ColumnTransformer

In [27]:
# ct bizim kendi verdiğiiz isimdir. 
# onehotencoder'ı yukarıda yarattık. [0] ise dönüşecek niteliklerin indeks değerleridir.

ct = ColumnTransformer(
    [('one_hot_encode_with_ct', onehotencoder, [3]),], 
    remainder='passthrough'
)

In [28]:
# ColumnTransformer nesnesi ile onehotencoder nesnesini eğitip gerekli dönüşümleri gerçekleştirip X'e tekrar atayalım.
# Burada X'in boyutu(shape) değişecektir.
X = ct.fit_transform(X)

In [30]:
X.shape

(50, 6)

In [31]:
# Toplam 4 niteliğimiz vardı 3 farklı şehir için gölge değişken oluştu 7. 
# LabelEncoder ile oluşan nitelik düştü kaldı 6. Hesap doğru

In [32]:
X[:3,:]

array([[0.0, 1.0, 0.0, 165349.2, 136897.8, 471784.1],
       [1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [0.0, 0.0, 1.0, 153441.51, 101145.55, 407934.54]], dtype=object)

## 2.6. Gölge Değişken Tuzağından (Dummy variable trap) Kaçınma 

In [33]:
X = X[:,0:]
X.shape

(50, 5)

## 2.7. Hedef değişken için Label Encoder

In [74]:
# Hedef niteliği zaten rakam olduğu için kodlamaya gerek yok. Ayrıca problem regresyon problemi

## 2.8. Standardizasyon FeatureScaling

In [75]:
# Harcamalar aynı türden yapmayabiliriz.

## 2.9. Veri setini train-test olarak ayırma (yatay bölme)

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
            test_size=0.25, random_state = 142)

In [35]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(37, 5)
(13, 5)
(37,)
(13,)


# 3. Model Oluşturma ve Eğitim

## 3.1. Lineer Regresyon Nesnesi Oluşturma (Boş model yaratma)

In [36]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

## 3.2. Lineer modeli eğitme (modeli şekillendirme - veriye uydurma - eğitme)

In [37]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## 3.3. Eğitilmiş modelle (makine) test seti kullanarak tahmin (prediction) 

In [38]:
y_pred = regressor.predict(X_test)

## 3.4. Tahmin sonuçlarını numpy.ndarray'den dataframe yapma (İncelemek daha Kolay)

In [39]:
predictions = pd.DataFrame(data=y_pred,    # values
                index=range(len(y_pred)),    # 1st column as index
                   columns=['y_pred'])  # 1st row as the column names

# Sadece y_pred'den oluşan df'e test(gerçek) y_test'i sütun olarak ekleme
predictions['y_test'] = y_test
predictions.head()

y_pred     y_test
0  159813.247738  155752.60
1  136034.415551  146121.95
2   74041.732845   90708.19
3   66245.848000   65200.33
4  172619.073397  182901.99

## 3.5. Regression metrics

### 3.5.1. explained_variance

In [40]:
# The sklearn.metrics module implements several loss, score, and utility functions to measure 
# regression performance. Some of those have been enhanced to handle the multioutput case: 
# mean_squared_error, mean_absolute_error, explained_variance_score and r2_score.

In [41]:
from sklearn.metrics import explained_variance_score

In [42]:
explained_variance_score(y_true=y_test, y_pred=y_pred)

0.9646473388561609

### 3.5.2. mean_absolute_error

In [43]:
from sklearn.metrics import mean_absolute_error

In [46]:
mean_absolute_error(y_pred=y_pred, y_true=y_test)

7481.044207153125

In [47]:
import math

In [48]:
# rmse
math.sqrt(mean_absolute_error(y_pred=y_pred, y_true=y_test))

86.49302981832193

### 3.5.3. r2_score 

In [49]:
from sklearn.metrics import r2_score

In [50]:
r2_score(y_test, y_pred)

0.9646383696314051

# 4. Model Anlamlılığı

In [51]:
# Model anlamlılığına ve girdi değişkenlerin modele katkısına bakmak için scikit-learn'den farklı
# bir kütüphane kullanacağız. Çünkü bunu scikit learn'da yapamıyoruz. 

In [52]:
import statsmodels.api as sm

## 4.1. Sabit ekleme

In [53]:
# Sabit Ekleme (aslında sbitin önünde bir tane x üzeri 0 var. 
# Onu temsilen yeni bir sütunu 1'ler ile dolduruyoruz)

In [54]:
X.shape

(50, 5)

In [55]:
X = sm.add_constant(X)
X.shape

(50, 6)

In [56]:
X[:5,:]

array([[1.0, 1.0, 0.0, 165349.2, 136897.8, 471784.1],
       [1.0, 0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [1.0, 0.0, 1.0, 153441.51, 101145.55, 407934.54],
       [1.0, 1.0, 0.0, 144372.41, 118671.85, 383199.62],
       [1.0, 0.0, 1.0, 142107.34, 91391.77, 366168.42]], dtype=object)

    Hatayı engellemek için aşağıdaki işlemi yapıyoruz.
    TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [57]:
X_opt = np.array(X, dtype=float)

In [58]:
import statsmodels.formula.api as sm
ols_obj = sm.OLS(endog=y, exog=X_opt)

In [59]:
ols_model = ols_obj.fit()

In [60]:
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Fri, 31 May 2019   Prob (F-statistic):           1.34e-27
Time:                        08:55:05   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.000    3.62e+04     6.4e+04
x1           -41.8870   3256.039     -0.013      0.990   -6604.003    6520.229
x2           198.7888   3371.007      0.059      0.953   -6595.030    6992.607
x3             0.8060      0.046     17.369      0.000       0.712       0.900
x4            -0.0270      0.052     -0.517      0.608      -0.132       0.078
x5             0.0270      0.017      1.574      0.123      -0.008       0.062
==============================================================================
Omnibus:                       14.782   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.266
Skew:                          -0.948   Prob(JB):                     2.41e-05
Kurtosis:                       5.572   Cond. No.                     1.45e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.45e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [104]:
df.head()

ArgeHarcamasi  YonetimGiderleri  PazarlamaHarcamasi     Sehir        Kar
0      165349.20         136897.80           471784.10  Istanbul  192261.83
1      162597.70         151377.59           443898.53    Ankara  191792.06
2      153441.51         101145.55           407934.54   Kocaeli  191050.39
3      144372.41         118671.85           383199.62  Istanbul  182901.99
4      142107.34          91391.77           366168.42   Kocaeli  166187.94

# 5. Model Oluşturma (Geriye Doğru Eleme Yöntemi)

## 1. Tur

In [105]:
# Sabit -> X[0]
# x1 : şehir-1 -> X[1]
# x2 : şehir-2 -> X[2]
# x3 : ArgeHarcamasi -> X[3]
# x4 : YonetimGiderleri -> X[4]
# x5 : PazarlamaHarcamasi -> X[5]

In [106]:
# Demekki şehir niteliği karı açıklamada anlamsız ben bu niteliği modele hiç katmıyorum.
# Modele girecekler 0: Sabit, 3: Arge, 4: Yönetim, 5: Pazarlama

In [61]:
import statsmodels.formula.api as sm
X_opt = np.array(X[:, [0, 3,4,5]], dtype=float)
ols_obj = sm.OLS(endog=y, exog=X_opt)

In [62]:
ols_model = ols_obj.fit()

In [63]:
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     296.0
Date:                Fri, 31 May 2019   Prob (F-statistic):           4.53e-30
Time:                        08:56:54   Log-Likelihood:                -525.39
No. Observations:                  50   AIC:                             1059.
Df Residuals:                      46   BIC:                             1066.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.012e+04   6572.353      7.626      0.000    3.69e+04    6.34e+04
x1             0.8057      0.045     17.846      0.000       0.715       0.897
x2            -0.0268      0.051     -0.526      0.602      -0.130       0.076
x3             0.0272      0.016      1.655      0.105      -0.006       0.060
==============================================================================
Omnibus:                       14.838   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.442
Skew:                          -0.949   Prob(JB):                     2.21e-05
Kurtosis:                       5.586   Cond. No.                     1.40e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.4e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 2. Tur

In [124]:
# Sabit -> X[0]
# x1 : ArgeHarcamasi -> X[3]
# x2 : YonetimGiderleri -> X[4]
# x3 : PazarlamaHarcamasi -> X[5]

In [125]:
# YonetimGiderleri niteliği karı açıklamada anlamsız bu niteliği modelden çıkarıyoruz.
# Modele girecekler 0: Sabit, 3: Arge, 5: Pazarlama

In [64]:
import statsmodels.formula.api as sm
X_opt = np.array(X[:, [0, 3, 5]], dtype=float)
ols_obj = sm.OLS(endog=y, exog=X_opt)

In [65]:
ols_model = ols_obj.fit()

In [66]:
ols_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Fri, 31 May 2019   Prob (F-statistic):           2.16e-31
Time:                        08:58:05   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.000    4.16e+04    5.24e+04
x1             0.7966      0.041     19.266      0.000       0.713       0.880
x2             0.0299      0.016      1.927      0.060      -0.001       0.061
==============================================================================
Omnibus:                       14.677   Durbin-Watson:                   1.257
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               21.161
Skew:                          -0.939   Prob(JB):                     2.54e-05
Kurtosis:                       5.575   Cond. No.                     5.32e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.32e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [129]:
# Sabit -> X[0]
# x1 : ArgeHarcamasi -> X[3]
# x2 : PazarlamaHarcamasi -> X[5]

In [130]:
# Pazarlama harcaması çok az bir farkla anlamsız kalıyor o.06 > 0.05
# İstersek bunu da çıkarabiliriz veya kalabilir.